In [1]:
# I located my data on aws ec2 machine (in docker container).
# After you create a postgres docker container (postgres-dash) you can follow this steps to insert all your data to db.

# su postgres
# psql -U dashuser
# create database dashapp;
# \c dashapp
# create table animals (age_upon_outcome text ,animal_id text,animal_type text,breed text ,color text,date_of_birth timestamp ,datetime_ timestamp, monthyear timestamp,name text,outcome_subtype text,outcome_type text, sex_upon_outcome text);

In [2]:
import pandas as pd
import psycopg2

import yaml
import warnings

warnings.filterwarnings('ignore')


with open('connection.yaml', 'r') as file:
    infos = yaml.safe_load(file)

conn = psycopg2.connect(user= infos['user'],
                        password= infos['password'],
                        host= infos['host'],
                        database= infos['database'])


insert_query = """
        INSERT INTO animals (age_upon_outcome, animal_id, animal_type, breed, color, date_of_birth, datetime_, monthyear, name, outcome_subtype, outcome_type, sex_upon_outcome)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
def insert_animal_one(values):
    """Fonsiyon input olarak liste formatinda dogru sirada 12 deger alir"""
    conn = psycopg2.connect(user= infos['user'],
                            password= infos['password'],
                            host= infos['host'],
                            database= infos['database'])

    cursor = conn.cursor()

    cursor.execute(insert_query, tuple(values))

    conn.commit()

    cursor.close()
    print('Kaydetme islemi basarili.')
    print('1 kayit yapildi.')

def read_animals(query):
    df = pd.read_sql(query, conn)
    return df

def insert_animals_many(df):
    """Fonksiyon input olarak pandas dataframe alir."""
    
    conn = psycopg2.connect(user= infos['user'],
                            password= infos['password'],
                            host= infos['host'],
                            database= infos['database'])

    cursor = conn.cursor()

    for i in range(len(df)):
        cursor.execute(insert_query, tuple(df.iloc[i, :]))
        if i % 5000 == 0:
            print(i)
            conn.commit()
    conn.commit()

    cursor.close()
    print('Kaydetme islemi basarili.')
    print(f'{len(df)} kayit yapildi.')

In [3]:
df = read_animals('SELECT * FROM animals')
df

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime_,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome


In [4]:
df = pd.read_csv('aac_shelter_outcomes.csv')
df.head()

,age_upon_outcome,animal_id,animal_type,breed,color,date_of_birth,datetime,monthyear,name,outcome_subtype,outcome_type,sex_upon_outcome
0,2 weeks,A684346,Cat,Domestic Shorthair Mix,Orange Tabby,2014-07-07T00:00:00,2014-07-22T16:04:00,2014-07-22T16:04:00,NaN,Partner,Transfer,Intact Male
1,1 year,A666430,Dog,Beagle Mix,White/Brown,2012-11-06T00:00:00,2013-11-07T11:47:00,2013-11-07T11:47:00,Lucy,Partner,Transfer,Spayed Female
2,1 year,A675708,Dog,Pit Bull,Blue/White,2013-03-31T00:00:00,2014-06-03T14:20:00,2014-06-03T14:20:00,*Johnny,NaN,Adoption,Neutered Male
3,9 years,A680386,Dog,Miniature Schnauzer Mix,White,2005-06-02T00:00:00,2014-06-15T15:50:00,2014-06-15T15:50:00,Monday,Partner,Transfer,Neutered Male
4,5 months,A683115,Other,Bat Mix,Brown,2014-01-07T00:00:00,2014-07-07T14:04:00,2014-07-07T14:04:00,NaN,Rabies Risk,Euthanasia,Unknown


In [6]:
insert_animals_many(df)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
Kaydetme islemi basarili.
78256 kayit yapildi.


In [3]:
query = """
SELECT
    COUNT(*) as total_count,
    COUNT(DISTINCT animal_id) as unique_number_of_animals,
    ROUND(CAST(COUNT(*) AS decimal)/COUNT(DISTINCT animal_id), 3) as chance_come_more_than_1_time,
    COUNT(DISTINCT animal_type) as unique_animal_types,
    COUNT(DISTINCT outcome_type) as unique_outcome_types,
    COUNT(DISTINCT outcome_subtype) as unique_outcome_subtypes,
    COUNT(DISTINCT sex_upon_outcome) as unique_sex_upon_outcome
FROM animals
"""

read_animals(query)

,total_count,unique_number_of_animals,chance_come_more_than_1_time,unique_animal_types,unique_outcome_types,unique_outcome_subtypes,unique_sex_upon_outcome
0,78256,70855,1.104,5,10,20,6


In [4]:
query = """
SELECT
    animal_type,
    COUNT(DISTINCT animal_id)
FROM animals
GROUP BY 1
ORDER BY 2 DESC
"""

xx = read_animals(query)
xx

,animal_type,count
0,Dog,38067
1,Cat,28210
2,Other,4235
3,Bird,334
4,Livestock,9


In [5]:
query = """
SELECT
    animal_id,
    COUNT(animal_id)
FROM animals
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
"""

read_animals(query)

,animal_id,count
0,A706536,11
1,A718223,11
2,A721033,10
3,A716018,8
4,A694501,8
5,A616444,8
6,A701901,7
7,A670612,7
8,A683108,7
9,A671704,7


In [6]:
query = """
SELECT
    animal_type,
    CAST(EXTRACT(year from date_of_birth) as integer) as birth_year,
    COUNT(DISTINCT animal_id) --DISTINCT is important here
FROM animals
GROUP BY 1,2
ORDER BY 2 DESC, 3 DESC
--LIMIT 10
"""

df = read_animals(query)
df.head(10)

,animal_type,birth_year,count
0,Cat,2017,3759
1,Dog,2017,1894
2,Other,2017,180
3,Bird,2017,27
4,Cat,2016,4872
5,Dog,2016,4117
6,Other,2016,762
7,Bird,2016,67
8,Livestock,2016,1
9,Cat,2015,6229


In [7]:
query = """
SELECT
    animal_type,
    outcome_subtype,
    COUNT(*),
    COUNT(DISTINCT animal_id)
FROM animals
GROUP BY 1,2
ORDER BY 1, 4 DESC
LIMIT 20
"""

read_animals(query)

,animal_type,outcome_subtype,count,count
0,Bird,NaN,141,141
1,Bird,Partner,92,92
2,Bird,Suffering,72,72
3,Bird,Foster,17,17
4,Bird,Medical,8,8
5,Bird,Offsite,2,2
6,Bird,In Kennel,1,1
7,Bird,Underage,1,1
8,Cat,NaN,10780,10163
9,Cat,Partner,9486,9477


In [8]:
query = """
WITH base as
(SELECT
    animal_id,
    COUNT(DISTINCT outcome_subtype) as number_of_different_outcomes
FROM animals
GROUP BY 1)

SELECT
    *
FROM base
WHERE number_of_different_outcomes >= 2
ORDER BY number_of_different_outcomes DESC
"""

df = read_animals(query)
df.head(10)
# more than 1 outcomes observed in 2080 animals.

,animal_id,number_of_different_outcomes
0,A738858,3
1,A691732,3
2,A707046,3
3,A701901,3
4,A746917,3
5,A729807,3
6,A757136,3
7,A672396,3
8,A673466,3
9,A697794,3


In [9]:
query = """
WITH base as
(SELECT
    animal_id,
    COUNT(DISTINCT outcome_subtype) as number_of_different_outcomes
FROM animals
GROUP BY 1)

SELECT
    number_of_different_outcomes,
    COUNT(*)
FROM base
GROUP BY 1
ORDER BY 2 DESC
"""

read_animals(query)

# more than 1 outcomes observed in 2080 animals.

,number_of_different_outcomes,count
0,1,68775
1,2,2054
2,3,26


In [10]:
query = """
SELECT
    outcome_subtype,
    COUNT(DISTINCT animal_id) as count_of_animals
FROM animals
GROUP BY 1
ORDER BY 2 DESC
"""

read_animals(query)

# more than 1 outcomes observed in 2080 animals.

,outcome_subtype,count_of_animals
0,NaN,37176
1,Partner,19618
2,Foster,5495
3,SCRP,3151
4,Suffering,2513
5,Rabies Risk,2415
6,Snr,618
7,Aggressive,506
8,Offsite,366
9,In Kennel,343


In [11]:
query = """
SELECT
    outcome_type,
    COUNT(DISTINCT animal_id) as count_of_animals
FROM animals
GROUP BY 1
ORDER BY 2 DESC
"""

read_animals(query)

# more than 1 outcomes observed in 2080 animals.

,outcome_type,count_of_animals
0,Adoption,30329
1,Transfer,23337
2,Return to Owner,12603
3,Euthanasia,6075
4,Died,680
5,Disposal,306
6,Rto-Adopt,150
7,Missing,45
8,Relocate,16
9,NaN,12


In [12]:
read_animals("SELECT MIN(date_of_birth), MAX(date_of_birth) FROM animals")

,min,max
0,1991-12-11,2017-12-25


In [13]:
query = """
SELECT
            DATE_TRUNC('MONTH', date_of_birth) month_,
            COUNT(DISTINCT animal_id) as count_of_animals
        FROM animals
        GROUP BY 1
        ORDER BY 1 DESC
"""

dfs = read_animals(query)


query = """
SELECT
            DATE_TRUNC('MONTH', datetime_) month_,
            COUNT(DISTINCT animal_id) as count_of_animals
        FROM animals
        GROUP BY 1
        ORDER BY 1 DESC
"""

dfz = read_animals(query)

In [14]:
dfs.head()

,month_,count_of_animals
0,2017-12-01,31
1,2017-11-01,117
2,2017-10-01,269
3,2017-09-01,361
4,2017-08-01,454


In [83]:
import plotly.graph_objects as go

# Create random data with numpy
import numpy as np
fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=dfs['month_'], y=dfs['count_of_animals'],
mode='lines+markers',
name='birth_date'))
fig.add_trace(go.Scatter(x=dfz['month_'], y=dfz['count_of_animals'],
mode='lines+markers',
name='datetime_'))

fig.show()

In [15]:
    dfx = read_animals(f"""
        SELECT
            outcome_type,
            outcome_subtype,
            COUNT(DISTINCT animal_id)
        FROM animals
        GROUP BY 1, 2
        """)

In [16]:
import plotly.express as px

In [17]:
px.sunburst(dfx, path = ['outcome_type', 'outcome_subtype'], values = 'count')

In [21]:
df = read_animals('select * from animals')

In [22]:
conv = dict()
string_version = ['months', 'days', 'years', 'weeks', 'year', 'month']
target_version = ['* 30', '* 1', '* 365', '* 7', '* 365', '* 30']

for s, t in zip(string_version, target_version):
    conv[s] = t

In [23]:
def normalize_outcome_age(df):
    converted = []
    for i in df['age_upon_outcome']:
        for j in list(conv.keys()):
            i = i.replace(j, conv[j])
        try: 
            converted.append(eval(i))
        except:
            converted.append(0)

    df['days_upon_outcome'] = converted

    return df

In [24]:
converted = []
for i in df['age_upon_outcome']:
    for j in list(conv.keys()):
        i = i.replace(j, conv[j])
    try: 
        converted.append(eval(i))
    except:
        converted.append(0)

In [25]:
df['days_upon_outcome'] = converted

In [27]:
px.box(df['days_upon_outcome'])

In [28]:
DFS = read_animals("""
SELECT
    animal_type,
    outcome_type,
    COUNT(DISTINCT animal_id) count_
FROM animals
GROUP BY 1,2
ORDER BY 3 DESC
""")


px.bar(DFS, x= 'outcome_type', y='count_', color = 'animal_type', barmode='group', text='count_')

In [29]:
query = """
SELECT DISTINCT outcome_type FROM animals
"""

read_animals(query)

,outcome_type
0,NaN
1,Missing
2,Disposal
3,Adoption
4,Transfer
5,Rto-Adopt
6,Relocate
7,Return to Owner
8,Died
9,Euthanasia


In [31]:
query = """
SELECT
    DATE_TRUNC('WEEK', datetime_),
    outcome_type,
    COUNT(DISTINCT animal_id)
FROM animals
GROUP BY 1,2
ORDER BY 1
"""

DF = read_animals(query)


px.line(DF, x = 'date_trunc', y="count", color='outcome_type')
# I checked if there is any dirty data like `animal that came after it died`, but not.
DF.head()

,date_trunc,outcome_type,count
0,2013-09-30,Adoption,116
1,2013-09-30,Died,4
2,2013-09-30,Euthanasia,28
3,2013-09-30,Relocate,1
4,2013-09-30,Return to Owner,57


In [32]:
px.line(DF, x = 'date_trunc', y="count", color='outcome_type')